# Load pretrained model to predict species of plant

### Import libraries and required packages

#### descriptions
1. **pandas** : reading csv file.(e.g., ground-truth)
2. **torch** : load a model for prediction.
3. **os** : find out your directory path.

In [2]:
import pandas as pd
import numpy as np
import torch
import os
import onnx
from PIL import Image

/home/silence/.pyenv/versions/3.7.2/envs/python-3.7.2/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/silence/.pyenv/versions/3.7.2/envs/python-3.7.2/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/silence/.pyenv/versions/3.7.2/envs/python-3.7.2/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.i

/home/silence/.pyenv/versions/3.7.2/envs/python-3.7.2/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/silence/.pyenv/versions/3.7.2/envs/python-3.7.2/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/silence/.pyenv/versions/3.7.2/envs/python-3.7.2/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.d

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.






### Define the directory

#### define the path your model stays

In [3]:
CKPT_DIR = 'mdfmt'
CKPT_PATH = os.path.join(CKPT_DIR, 'best_model96.pt')

#### define the ground-truth path

In [4]:
LABEL_DIR = 'label2.csv'
label_df = pd.read_csv(LABEL_DIR)
label_df.head()

,dirpath,target,label
0,DaanForestPark/20180330/1lZsRrQzj,通泉草,66
1,DaanForestPark/20180330/4PdXwYcGt,紫花酢漿草,51
2,DaanForestPark/20180330/6VrmeiUE5,通泉草,66
3,DaanForestPark/20180330/7nrSoiuHL,通泉草,66
4,DaanForestPark/20180330/7RKdkJLgy,通泉草,66


#### define your img file

In [5]:
DATA_SAMPLE = 'sample/sample.jpg'

#### Load your .ckpt model

In [6]:
model = torch.load(CKPT_PATH)

### Predict the result when an image comes

In [7]:
# convert image to rgb-format numpy array
oneshot = Image.open(DATA_SAMPLE).convert('RGB')
# do dimension transpose: (height, weidth, channel) to (channel, height, weidth)
oneshot = torch.Tensor(np.asarray(oneshot)).permute(2,0,1).unsqueeze(0)
# input shape
print('shape of oneshot={}'.format(oneshot.shape))
pred = model(oneshot.cuda())
# output shape
print('shape of pred={}'.format(pred.shape))
# get a predicted label
_, predicted = torch.max(pred.cpu().data, 1)
print('predict label={}'.format(predicted.item()))

shape of oneshot=torch.Size([1, 3, 192, 340])
shape of pred=torch.Size([1, 85])
predict label=4


#### Show the mapping tag

In [8]:
res = label_df[label_df['label']==predicted.item()]['target'].unique()[0]
print('the predicted result is {}'.format(res))

the predicted result is 仙丹花


### Convert to ONNX

In [9]:
onnx_filename = 'best_model96.onnx'
dummy_input = torch.rand(1, 3, 224, 224).cuda()
# input_names = []
# for key, module in model._modules.items():
#     input_names.append("l_{}_".format(key) + module._get_name())
torch.onnx.export(model,
                  dummy_input,
                  onnx_filename,
                  input_names=['input'],
                  output_names=['output'],
                  verbose = True)

graph(%input : Float(1, 3, 224, 224),
      %conv1.0.weight : Float(24, 3, 3, 3),
      %conv1.1.weight : Float(24),
      %conv1.1.bias : Float(24),
      %conv1.1.running_mean : Float(24),
      %conv1.1.running_var : Float(24),
      %conv1.1.num_batches_tracked : Long(),
      %stage2.0.branch1.0.weight : Float(24, 1, 3, 3),
      %stage2.0.branch1.1.weight : Float(24),
      %stage2.0.branch1.1.bias : Float(24),
      %stage2.0.branch1.1.running_mean : Float(24),
      %stage2.0.branch1.1.running_var : Float(24),
      %stage2.0.branch1.1.num_batches_tracked : Long(),
      %stage2.0.branch1.2.weight : Float(58, 24, 1, 1),
      %stage2.0.branch1.3.weight : Float(58),
      %stage2.0.branch1.3.bias : Float(58),
      %stage2.0.branch1.3.running_mean : Float(58),
      %stage2.0.branch1.3.running_var : Float(58),
      %stage2.0.branch1.3.num_batches_tracked : Long(),
      %stage2.0.branch2.0.weight : Float(58, 24, 1, 1),
      %stage2.0.branch2.1.weight : Float(58),
      %stage2

### Convert to .pb

In [2]:
import onnx
from onnx_tf.backend import prepare
from PIL import Image
import numpy as np
onnx_filename = 'best_model96.onnx'
onnx_model = onnx.load(onnx_filename)
tf_rep = prepare(onnx_model,strict=False)

image = Image.open('sample/sample.jpg')
image = image.resize((224, 224),Image.ANTIALIAS)
img_np = np.asarray(image)[np.newaxis,:,:].transpose(0,3,1,2)

output_onnx_tf = tf_rep.run(img_np)
print('output_onnx_tf = {}'.format(output_onnx_tf))
# onnx --> tf.graph.pb
tf_pb_path = onnx_filename + '_graph.pb'
tf_rep.export_graph(tf_pb_path)

/home/silence/.pyenv/versions/3.7.2/envs/python-3.7.2/lib/python3.7/site-packages/onnx_tf/common/__init__.py:89: UserWarning: onnx_tf.common.get_outputs_names is deprecated. It will be removed in future release. Use TensorflowGraph.get_outputs_names instead.
  warnings.warn(message)


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.






/home/silence/.pyenv/versions/3.7.2/envs/python-3.7.2/lib/python3.7/site-packages/onnx_tf/common/handler_helper.py:37: UserWarning: Unknown op ConstantFill in domain `ai.onnx`.
  handler.ONNX_OP, handler.DOMAIN or "ai.onnx"))
/home/silence/.pyenv/versions/3.7.2/envs/python-3.7.2/lib/python3.7/site-packages/onnx_tf/common/handler_helper.py:34: UserWarning: Fail to get since_version of ConvInteger in domain `` with max_inclusive_version=9. Set to 1.
  handler.ONNX_OP, handler.DOMAIN, version))
/home/silence/.pyenv/versions/3.7.2/envs/python-3.7.2/lib/python3.7/site-packages/onnx_tf/common/handler_helper.py:34: UserWarning: Fail to get since_version of DequantizeLinear in domain `` with max_inclusive_version=9. Set to 1.
  handler.ONNX_OP, handler.DOMAIN, version))
/home/silence/.pyenv/versions/3.7.2/envs/python-3.7.2/lib/python3.7/site-packages/onnx_tf/common/handler_helper.py:34: UserWarning: Fail to get since_version of GatherND in domain `` with max_inclusive_version=9. Set to 1.
  ha

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
Instructions for updating:
Use keras.layers.flatten instead.
output_onnx_tf = Outputs(output=array([[ -745.68964 ,  -316.87534 ,  -453.9025  ,  -432.44373 ,
         -628.7093  ,  -304.1578  ,  -672.621   ,    93.88486 ,
         -277.25665 ,  -395.79486 ,  -454.3425  ,  -770.14264 ,
          -26.148468,    47.224968,  -584.1148  ,   103.73132 ,
         -274.15347 ,  -164.76045 ,    70.65137 ,  -723.9883  ,
         -142.9099  ,  -103.91418 ,  -281.58218 ,   -89.67374 ,
         -296.01285 ,  -418.65158 ,  -242.62122 ,  -690.9727  ,
        -1026.5464  ,  -391.1395  ,  -303.26697 ,  -442.6496  ,
         -951.9504  ,  -198.8128  ,  -771.601   ,  -374.12582 ,
         -596.47736 ,  -829.7157  ,  -555.22046 ,  -613.347   ,
           94.0568  ,  -456.36063 ,  -142.92284 ,  -744.7144  ,
         -

### Convert to tflite

In [3]:
import tensorflow as tf
input_arrays = ['input']
output_arrays = ['output']
# Converting a SavedModel.
converter = tf.lite.TFLiteConverter.from_frozen_graph(tf_pb_path, input_arrays, output_arrays)
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

6464064

In [4]:
interpreter = tf.lite.Interpreter(model_path="converted_model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [5]:
input_details

[{'name': 'input',
  'index': 7995,
  'shape': array([  1,   3, 224, 224], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0)}]

In [6]:
output_details

[{'name': 'output',
  'index': 7997,
  'shape': array([], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0)}]

### Convert to mlmodel

In [8]:
from onnx_coreml import convert
# Load the ONNX model as a CoreML model
model = convert(model='model_best.onnx',minimum_ios_deployment_target='13')
# Save the CoreML model
model.save('model_best.mlmodel')

1/69: Converting Node Type Conv
2/69: Converting Node Type BatchNormalization
3/69: Converting Node Type Relu
4/69: Converting Node Type MaxPool
5/69: Converting Node Type Conv
6/69: Converting Node Type BatchNormalization
7/69: Converting Node Type Relu
8/69: Converting Node Type Conv
9/69: Converting Node Type BatchNormalization
10/69: Converting Node Type Add
11/69: Converting Node Type Relu
12/69: Converting Node Type Conv
13/69: Converting Node Type BatchNormalization
14/69: Converting Node Type Relu
15/69: Converting Node Type Conv
16/69: Converting Node Type BatchNormalization
17/69: Converting Node Type Add
18/69: Converting Node Type Relu
19/69: Converting Node Type Conv
20/69: Converting Node Type BatchNormalization
21/69: Converting Node Type Relu
22/69: Converting Node Type Conv
23/69: Converting Node Type BatchNormalization
24/69: Converting Node Type Conv
25/69: Converting Node Type BatchNormalization
26/69: Converting Node Type Add
27/69: Converting Node Type Relu
28/69:

In [ ]:
def print_inputs(pb_filepath):
    with tf.gfile.GFile(pb_filepath, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

        with tf.Graph().as_default() as graph:
            tf.import_graph_def(graph_def, name='')

        input_list = []
        for op in graph.get_operations(): # tensorflow.python.framework.ops.Operation
            if op.type == "Placeholder":
                input_list.append(op.name)

        print('Inputs:', input_list)


def print_outputs(pb_filepath):
    with open(pb_filepath, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

        name_list = []
        input_list = []
        for node in graph_def.node: # tensorflow.core.framework.node_def_pb2.NodeDef
            name_list.append(node.name)
            input_list.extend(node.input)

        outputs = set(name_list) - set(input_list)
        print('Outputs:', list(outputs))

if __name__ == '__main__':
    print_inputs(tf_pb_path)
    print_outputs(tf_pb_path)